# Models & CRUD Operations

**Use case:** Managing an e-commerce product catalog.

This notebook covers the fundamentals of defining models and performing Create,
Read, Update, and Delete operations with SurrealDB-ORM. If you have used
Django's ORM before, the patterns will feel familiar.

## Prerequisites

A SurrealDB Docker container must be running. See `00_setup.ipynb` for
instructions on starting the database and configuring environment variables.

In [1]:
# -- Setup (run this cell first) ------------------------------------------
import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)
print("Connection configured.")

Connection configured.


## Defining Models

Models are Python classes that extend `BaseSurrealModel`. Each model maps to a
SurrealDB table. Fields are declared as type-annotated class attributes -- just
like Pydantic models (because they *are* Pydantic models under the hood).

Two configuration options you'll use often:

- **`table_name`** -- Override the default table name (which is the class name
  in lowercase).
- **`primary_key`** -- Tell the ORM which field to use as the SurrealDB record
  ID. By default, the `id` field is used.

In [3]:
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict


class Product(BaseSurrealModel):
    """A product in the catalog. Uses an explicit table name."""

    model_config = SurrealConfigDict(table_name="products")

    id: str | None = None
    name: str
    price: float
    category: str = "general"
    in_stock: bool = True


class Customer(BaseSurrealModel):
    """A customer account. Uses `email` as the primary key."""

    model_config = SurrealConfigDict(
        table_name="customers",
        primary_key="email",
    )

    name: str
    email: str
    age: int = 0


print(f"Product table: {Product.get_table_name()}")
print(f"Customer table: {Customer.get_table_name()}")

Product table: products
Customer table: customers


## Creating Records -- `save()`

Call `save()` on a model instance to persist it to SurrealDB. If the record
already exists (same ID), it performs an upsert. If the `id` is `None`,
SurrealDB generates a random ID automatically.

In [4]:
# Create products -- SurrealDB will auto-generate IDs
laptop = Product(name="Laptop Pro 16", price=1599.99, category="electronics")
await laptop.save()
print(f"Created: {laptop.name} (id={laptop.id})")

headphones = Product(name="Wireless Headphones", price=79.99, category="electronics")
await headphones.save()
print(f"Created: {headphones.name} (id={headphones.id})")

notebook = Product(name="Leather Notebook", price=24.50, category="stationery")
await notebook.save()
print(f"Created: {notebook.name} (id={notebook.id})")

# Create a product with an explicit ID
pen = Product(id="pen_001", name="Fountain Pen", price=45.00, category="stationery")
await pen.save()
print(f"Created: {pen.name} (id={pen.id})")

Created: Laptop Pro 16 (id=41ts5tlm21196ad4va4k)
Created: Wireless Headphones (id=6i3y2vu0ktso1fbcfpl3)
Created: Leather Notebook (id=simis5a1j0vylnk2saq9)
Created: Fountain Pen (id=pen_001)


In [5]:
# Create customers -- email is the primary key
alice = Customer(name="Alice", email="alice@example.com", age=30)
await alice.save()
print(f"Created customer: {alice.name} ({alice.email})")

bob = Customer(name="Bob", email="bob@example.com", age=25)
await bob.save()
print(f"Created customer: {bob.name} ({bob.email})")

Created customer: Alice (alice@example.com)
Created customer: Bob (bob@example.com)


## Reading Records -- `get()` and `all()`

Use `objects().get(id)` to fetch a single record by ID. Use `objects().all()`
to fetch every record in a table.

Both return fully-typed Pydantic model instances, so you get autocomplete and
validation for free.

In [6]:
# Fetch a single product by its explicit ID
fetched_pen = await Product.objects().get("pen_001")
print(f"Fetched: {fetched_pen.name} -- ${fetched_pen.price}")

# Fetch by full SurrealDB record format (also works)
fetched_pen2 = await Product.objects().get("products:pen_001")
print(f"Same product: {fetched_pen2.name}")

Fetched: Fountain Pen -- $45.0
Same product: Fountain Pen


In [7]:
# Fetch all products in the table
all_products = await Product.objects().all()
print(f"Total products: {len(all_products)}")
for p in all_products:
    print(f"  - {p.name}: ${p.price} [{p.category}]")

Total products: 4
  - Laptop Pro 16: $1599.99 [electronics]
  - Wireless Headphones: $79.99 [electronics]
  - Fountain Pen: $45.0 [stationery]
  - Leather Notebook: $24.5 [stationery]


## Updating Records -- `save()` and `merge()`

There are two ways to update a record:

1. **`save()`** -- Modify the instance attributes and call `save()` again. For
   records that already exist, this performs a partial merge (only sending
   changed fields).

2. **`merge(**fields)`** -- Explicitly merge specific fields. This is useful
   when you want to update a few fields without loading the full object first.
   Pass `refresh=False` to skip the round-trip SELECT after the update.

In [8]:
# Method 1: Modify and save
fetched_pen.price = 49.99
await fetched_pen.save()
print(f"Updated price via save(): ${fetched_pen.price}")

# Method 2: Partial merge -- only sends the specified fields
await fetched_pen.merge(in_stock=False)
print(f"Updated in_stock via merge(): {fetched_pen.in_stock}")

# Fire-and-forget merge (skip the extra SELECT round-trip)
await fetched_pen.merge(category="writing", refresh=False)
print("Merged category without refresh.")

Updated price via save(): $49.99
Updated in_stock via merge(): False
Merged category without refresh.


## Deleting Records -- `delete()`

Call `delete()` on a model instance to remove it from the database.

In [9]:
# Delete the pen product
await fetched_pen.delete()
print("Deleted the fountain pen.")

# Verify it's gone
remaining = await Product.objects().all()
print(f"Remaining products: {len(remaining)}")
for p in remaining:
    print(f"  - {p.name}")

Deleted the fountain pen.
Remaining products: 3
  - Laptop Pro 16
  - Wireless Headphones
  - Leather Notebook


## Handling Missing Records -- `DoesNotExist`

When `get()` can't find a record, it raises `Model.DoesNotExist` (similar to
Django). Always wrap lookups in a try/except when the record might not exist --
for example, when the ID comes from user input.

In [10]:
# Try to fetch a record that was deleted
try:
    ghost = await Product.objects().get("pen_001")
except Product.DoesNotExist:
    print("Product not found -- DoesNotExist raised as expected.")

# This works for any model
try:
    unknown = await Customer.objects().get("nobody@example.com")
except Customer.DoesNotExist:
    print("Customer not found -- DoesNotExist raised as expected.")

Product not found -- DoesNotExist raised as expected.
Customer not found -- DoesNotExist raised as expected.


## Custom Table Names and Primary Keys

`SurrealConfigDict` controls how a model maps to SurrealDB. Here are the most
commonly used options:

| Option          | Description                                      | Default           |
|-----------------|--------------------------------------------------|-------------------|
| `table_name`    | Override the SurrealDB table name                | Class name (lower)|
| `primary_key`   | Field to use as the record ID                    | `"id"`            |
| `server_fields` | Fields populated server-side (excluded from save)| `[]`              |
| `table_type`    | Table classification (NORMAL, USER, RELATION)    | `NORMAL`          |
| `schema_mode`   | SCHEMAFULL or SCHEMALESS                         | `SCHEMAFULL`      |

The `Customer` model above uses `primary_key="email"`, so the record ID in
SurrealDB is the email address itself (e.g., `customers:alice@example.com`).

In [11]:
# The Customer primary key is the email field
alice_fetched = await Customer.objects().get("alice@example.com")
print(f"Fetched by email PK: {alice_fetched.name} ({alice_fetched.email})")

# You can inspect the table name at runtime
print(f"\nProduct table:  {Product.get_table_name()}")
print(f"Customer table: {Customer.get_table_name()}")

Fetched by email PK: Alice (alice@example.com)

Product table:  products
Customer table: customers


## Cleanup

Remove the tables we created so the database is clean for other notebooks.

In [12]:
# Delete all records from the tables used in this notebook
client = await SurrealDBConnectionManager.get_client()
await client.query("REMOVE TABLE products;")
await client.query("REMOVE TABLE customers;")
print("Cleanup complete -- tables removed.")

Cleanup complete -- tables removed.
